In [1]:
%pwd

'/home/malik-harris/dental/research'

In [2]:
import os

In [3]:
os.chdir("../")

In [4]:
%pwd

'/home/malik-harris/dental'

In [5]:
import json
import pandas as pd
from collections import Counter

In [6]:
ann_dir = "data/Raw/ann"
img_dir = "data/Raw/img"

records = []


In [7]:
for fname in os.listdir(ann_dir):
    if fname.endswith(".json"):
        with open(os.path.join(ann_dir, fname), "r") as f:
            data = json.load(f)
        # Assuming 'objects' is a list of annotations in the JSON
        class_titles = [obj['classTitle'] for obj in data.get('objects', [])]
        class_count = Counter(class_titles)
        record = {"image": fname.replace(".json", "")}
        record.update(class_count)
        records.append(record)


In [8]:

df = pd.DataFrame(records)
df = df.fillna(0).astype({col: int for col in df.columns if col != "image"})
df

,image,Tooth,Caries,Cavity,Crack
0,2527_jpg.rf.d5bdec9981c956eb6eea3c08ad367e24.jpg,13,0,0,0
1,2067_jpg.rf.4cc1ec41b91ea2d5f53ef1d47c5b12b8.jpg,13,0,0,0
2,905_jpg.rf.fb2c3991f7efcec72de5d25e19f0a17a.jpg,13,0,0,0
3,1488_jpg.rf.8989698be8fe61fcfcecba136a37021c.jpg,2,5,0,0
4,1007_jpg.rf.d08c6d48aad78ebc1ae9f48075dc628a.jpg,10,0,1,0
...,...,...,...,...,...
2490,1485_jpg.rf.61be5c1599a2d053201de51775cfadfe.jpg,15,4,0,0
2491,79_jpg.rf.a2f9945c68d58280699e85a244df8180.jpg,10,1,1,0
2492,1707_jpg.rf.0a703239bb3511643f398953ebd3249c.jpg,6,0,0,0
2493,2031_jpg.rf.f62e7de000956cd9532406ba979ea3b0.jpg,6,0,0,0


In [9]:
import shutil

In [10]:
# Identify all class columns except 'image' and 'Tooth'
class_cols = [col for col in df.columns if col not in ['image', 'Tooth']]

# Select rows where all other classes are 0 and 'Tooth' > 0
tooth_only = df[(df[class_cols].sum(axis=1) == 0) & (df['Tooth'] > 0)]

# Create the target directory if it doesn't exist
target_dir = "data/final/Normal"
os.makedirs(target_dir, exist_ok=True)

# Move the corresponding images
for img_name in tooth_only["image"]:
    src = os.path.join(img_dir, img_name)
    if os.path.exists(src):
        shutil.move(src, os.path.join(target_dir, img_name))

In [11]:
import shutil

# Only consider these classes (exclude 'Tooth')
target_classes = [col for col in df.columns if col not in ['image', 'Tooth']]

base_dir = "data/final"
os.makedirs(base_dir, exist_ok=True)

for idx, row in df.iterrows():
    # Find target classes with count > 0
    present_classes = [col for col in target_classes if row[col] > 0]
    if not present_classes:
        continue  # skip if no target class present

    # Create directory name: single class or combined (sorted, joined by '_')
    dir_name = "_".join(sorted(present_classes))
    target_dir = os.path.join(base_dir, dir_name)
    os.makedirs(target_dir, exist_ok=True)

    # Move the image if it exists
    src = os.path.join(img_dir, row["image"])
    if os.path.exists(src):
        shutil.move(src, os.path.join(target_dir, row["image"]))

In [12]:
import os

final_dir = "data/final"
class_counts = {}

for class_dir in os.listdir(final_dir):
    dir_path = os.path.join(final_dir, class_dir)
    if os.path.isdir(dir_path):
        count = len([f for f in os.listdir(dir_path) if os.path.isfile(os.path.join(dir_path, f))])
        class_counts[class_dir] = count

# Display the counts
for class_name, count in class_counts.items():
    print(f"{class_name}: {count} images")

Caries_Cavity_Crack: 15 images
Caries_Cavity: 686 images
Crack: 91 images
Caries: 452 images
Cavity_Crack: 11 images
Normal: 916 images
Caries_Crack: 37 images
Cavity: 287 images
